In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense,TextVectorization, Embedding, Flatten, Dropout

from sklearn.model_selection import train_test_split

2024-11-09 17:23:40.622759: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
samples = pd.read_csv('Reviews.csv')

In [3]:
def standardize(input):
    result = tf.strings.lower(input)
    result = tf.strings.regex_replace(result, r"[^a-z\s]", " ")
    result = tf.strings.regex_replace(result, r"\s+", " ")
    return result

In [4]:
vectorizer = TextVectorization(max_tokens=4096, output_mode="int", output_sequence_length=2048, standardize=standardize)
vectorizer.adapt([samples['Review']])

In [5]:
x = np.array(samples['Review'].tolist())
y = np.array(samples['Sentiment'].tolist())

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((40000,), (40000,), (10000,), (10000,))

In [6]:
train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(256)
test = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(256)

In [7]:
model = Sequential([
    vectorizer,
    Embedding(input_dim=4096, output_dim=32),
    Flatten(),
    Dropout(.25),
    Dense(1, activation='sigmoid')
])

In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train, epochs=64, validation_data=test)

Epoch 1/64
157/157 [==============================] - 45s 271ms/step - loss: 0.5970 - accuracy: 0.6686 - val_loss: 0.3900 - val_accuracy: 0.8473
Epoch 2/64
157/157 [==============================] - 47s 300ms/step - loss: 0.3199 - accuracy: 0.8720 - val_loss: 0.2950 - val_accuracy: 0.8788
Epoch 3/64
157/157 [==============================] - 42s 265ms/step - loss: 0.2550 - accuracy: 0.9011 - val_loss: 0.2768 - val_accuracy: 0.8876
Epoch 4/64
157/157 [==============================] - 42s 270ms/step - loss: 0.2242 - accuracy: 0.9140 - val_loss: 0.2740 - val_accuracy: 0.8879
Epoch 5/64
157/157 [==============================] - 42s 267ms/step - loss: 0.2020 - accuracy: 0.9236 - val_loss: 0.2791 - val_accuracy: 0.8862
Epoch 6/64
157/157 [==============================] - 42s 266ms/step - loss: 0.1813 - accuracy: 0.9334 - val_loss: 0.2792 - val_accuracy: 0.8878
Epoch 7/64
157/157 [==============================] - 41s 264ms/step - loss: 0.1603 - accuracy: 0.9438 - val_loss: 0.2835 - val_ac